In [2]:
%load_ext autoreload


In [3]:
%autoreload 2

In [4]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime
''' Sn properties - Masses, Ratios, Cup Configurations, ... '''
from sn_config import *
'''Classes for Reading in the Data and applying Internal Normalisation'''
from nu_data_reduction import NU_data_read, int_norm, evaluation
from pylab import plot, show, savefig
from scipy import stats
import os.path
import re
#%pylab inline

In [5]:
path = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2015-03-25/"
path_sav = "/Volumes/friebelm/PhD/NU Plasma/Measurements/compilation/"
path_all = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2016-05-19/"
files_start = 2336
files_end = 2337


In [6]:
# cup configuration
#cup_config = cycle_Sb
cup_config = cycles2
# Isotopes used for Interference correction
corr_isotopes_1 = {"Cd" : "111", "Te" : "126"}
corr_isotopes_2 = {"Cd" : "111","Te" : "125", "Xe" : "129"}
corr_isotopes_Sb = {"Te": "125"}
# Mass Range of cup configuration
mass_range = cycles2_mass_range
#isotopes = [["111", "112", "114", "115", "116", "117", "118", "119", "122", "124"], ["117", "118", "119", "122", "124"]]
isotopes = [["111", "112", "114", "115", "116", "117", "118", "119", "122", "124", "125", "126", "129"]]
#isotopes = [["116", "117", "118", "119", "122", "124"]]
denom_isotope = "120"
isotope_column = ["111", "112", "114", "115", "116", "117", "118", "119", "122", "124", "125", "126", "129"]
data_sample_column = [(i + "/" + "120") for i in isotope_column]
print data_sample_column
#number of iterations for beta
iter_beta = 10

# Interference_corr on the denominator isotope
isotope_denom_corr = True

blk_corr = True

['111/120', '112/120', '114/120', '115/120', '116/120', '117/120', '118/120', '119/120', '122/120', '124/120', '125/120', '126/120', '129/120']


In [7]:
filenames = os.listdir(path_all)

In [8]:
blk_ls = []
sample_ls = []
std_ls = []
for i in filenames:
    sample = re.search(r""+"Data_"+"(.*).csv", i)
    if sample == None:
        None
    else:
        sample = sample.group(1)
        df = NU_data_read(path_all, sample, cup_config)
        sample_name = df.extract_metadata(sample, "Sample Name")
        if sample_name == "blank sol" or sample_name == "wash" or sample_name == "wash clean":
            blk_ls.append(sample)
        elif sample_name == "SPEX 100ppb" or sample_name == "NIST 100ppb":
            std_ls.append(sample)
        else:
            sample_ls.append(sample)

In [9]:
columns = ["cycle", "sample", "date","H10 (1)", "H9 (1)", "H8 (1)", "H7 (1)", "H6 (1)", "H5 (1)", "H4 (1)", "H3 (1)", "H2 (1)", "H1 (1)", "Ax (1)", "L1 (1)", "L2 (1)", "L3 (1)", "L4 (1)", 
             "H10 (2)", "H9 (2)", "H8 (2)", "H7 (2)", "H6 (2)", "H5 (2)", "H4 (2)", "H3 (2)", "H2 (2)", "H1 (2)", "Ax (2)", "L1 (2)", "L2 (2)", "L3 (2)", "L4 (2)"]
df_all = pd.DataFrame(columns = columns)
for sample in blk_ls:
    df = NU_data_read(path_all, sample, cup_config)
    datafile = "Data_" + str(sample) + ".csv"
    data = pd.read_csv(path_all + datafile, skiprows=58, index_col='Cycle')
    if "H8 (2)" in data.columns:
        data_zero = data.ix[:,"H10 (Z1)":"L4 (Z1)"]
        data_1 = data.ix[:,"H10 (1)":"L4 (1)"]
        data_2 = data.ix[:,"H10 (2)":"L4 (2)"]
        data_zero.columns = data_1.columns
        data_1_corr = data_1 - data_zero
        data_1_corr["cycle"] = data_1_corr.index.values
        data_zero.columns = data_2.columns
        data_2_corr = data_2 - data_zero
        data_2_corr["cycle"] = data_2_corr.index.values
        data_1_corr = data_1_corr.merge(data_2_corr)
    else:
        data_zero = data.ix[:,"H10 (Z1)":"L4 (Z1)"]
        data_1 = data.ix[:,"H10 (1)":"L4 (1)"]
        data_zero.columns = data_1.columns
        data_1_corr = data_1 - data_zero
        data_1_corr["cycle"] = data_1_corr.index.values
        
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    data_1_corr["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_1_corr["sample"] = sample_name
    df_all = pd.concat([df_all,data_1_corr], axis=0, ignore_index=True)
    #df_all = df_all.append(data_1_corr)
    df_all = df_all[columns]
df_all.set_index("date", inplace = True)
df_all

,cycle,sample,H10 (1),H9 (1),H8 (1),H7 (1),H6 (1),H5 (1),H4 (1),H3 (1),...,H5 (2),H4 (2),H3 (2),H2 (2),H1 (2),Ax (2),L1 (2),L2 (2),L3 (2),L4 (2)
date,,,,,,,,,,,,,,,,,,,,,
2016-05-19 15:09:00,1,blank sol,0,0,0.001513,0.001232,0.009708,0.001997,0.006525,0.002016,...,0.000102,1.147750e-05,0.001101,0.000070,0.000840,0.000093,0.007013,0.001489,0.004183,0.001330
2016-05-19 17:06:00,1,blank sol,0,0,0.000759,0.000551,0.004992,0.001005,0.002813,0.000878,...,0.000112,2.461620e-05,0.000721,0.000029,0.000551,0.000071,0.004818,0.000977,0.002709,0.000859
2016-05-19 17:06:00,2,blank sol,0,0,0.001544,0.001161,0.009221,0.002091,0.005852,0.001826,...,0.000068,3.386470e-05,0.000848,0.000065,0.000626,0.000062,0.005588,0.001147,0.003251,0.001017
2016-05-19 17:06:00,3,blank sol,0,0,0.001221,0.000912,0.007613,0.001691,0.004740,0.001468,...,0.000105,-4.691600e-06,0.000816,0.000071,0.000596,0.000072,0.005277,0.001083,0.003032,0.000977
2016-05-19 17:06:00,4,blank sol,0,0,0.001046,0.000780,0.006581,0.001418,0.003951,0.001248,...,0.000071,4.460150e-05,0.000788,0.000072,0.000596,0.000059,0.005252,0.001056,0.003005,0.000952
2016-05-19 17:06:00,5,blank sol,0,0,0.002443,0.001879,0.014018,0.003363,0.009351,0.002949,...,0.000072,2.882240e-05,0.000645,0.000060,0.000475,0.000058,0.004467,0.000872,0.002412,0.000748
2016-05-19 17:06:00,6,blank sol,0,0,0.000695,0.000512,0.004720,0.000935,0.002608,0.000822,...,0.000085,4.033270e-05,0.001219,0.000043,0.000935,0.000079,0.007643,0.001706,0.004729,0.001458
2016-05-19 17:06:00,7,blank sol,0,0,0.001375,0.001024,0.008292,0.001869,0.005157,0.001609,...,0.000070,-5.411200e-06,0.000948,0.000049,0.000696,0.000058,0.005928,0.001218,0.003505,0.001124
2016-05-19 17:06:00,8,blank sol,0,0,0.000689,0.000482,0.004682,0.000917,0.002555,0.000803,...,0.000110,2.747650e-05,0.000714,0.000064,0.000525,0.000066,0.004729,0.000915,0.002625,0.000865


In [10]:
def mean(x):
    mean = np.nanmean(x) 
    return mean

df_all_mean = df_all.drop("cycle", axis=1).groupby(df_all.index).agg(mean)
df_all_mean["sample"] = df_all[df_all["cycle"] == 1]["sample"]
df_all_mean

,H10 (1),H9 (1),H8 (1),H7 (1),H6 (1),H5 (1),H4 (1),H3 (1),H2 (1),H1 (1),...,H4 (2),H3 (2),H2 (2),H1 (2),Ax (2),L1 (2),L2 (2),L3 (2),L4 (2),sample
date,,,,,,,,,,,,,,,,,,,,,
2016-05-19 15:09:00,0,0,0.001513,0.001232,0.009708,0.001997,0.006525,0.002016,0.004293,0.000118,...,0.000011,0.001101,0.000070,0.000840,0.000093,0.007013,0.001489,0.004183,0.001330,blank sol
2016-05-19 16:42:00,0,0,0.000867,0.000635,0.005609,0.001149,0.003216,0.001021,0.002642,0.000075,...,0.000016,0.000977,0.000055,0.000724,0.000076,0.006199,0.001318,0.003674,0.001159,blank sol
2016-05-19 17:06:00,0,0,0.001320,0.001003,0.008047,0.001809,0.005027,0.001578,0.003700,0.000091,...,0.000021,0.001695,0.000058,0.001298,0.000071,0.010090,0.002328,0.006507,0.002045,blank sol
2016-05-19 17:37:00,0,0,0.000733,0.000523,0.004853,0.000966,0.002710,0.000860,0.002221,0.000066,...,0.000010,0.000645,0.000060,0.000473,0.000078,0.004423,0.000863,0.002406,0.000770,blank sol
2016-05-19 18:03:00,0,0,0.000872,0.000648,0.005619,0.001178,0.003289,0.001037,0.002644,0.000069,...,0.000010,0.001065,0.000049,0.000802,0.000061,0.006674,0.001453,0.004043,0.001284,blank sol
2016-05-19 19:23:00,0,0,0.000863,0.000663,0.005634,0.001180,0.003340,0.001058,0.002563,0.000061,...,0.000018,0.000885,0.000061,0.000656,0.000075,0.005646,0.001202,0.003357,0.001055,blank sol
2016-05-19 20:43:00,0,0,0.000872,0.000638,0.005580,0.001191,0.003341,0.001053,0.002620,0.000058,...,0.000004,0.000941,0.000049,0.000702,0.000062,0.005932,0.001285,0.003616,0.001140,blank sol
2016-05-19 22:03:00,0,0,0.001183,0.000902,0.007222,0.001635,0.004566,0.001436,0.003380,0.000087,...,0.000007,0.001159,0.000044,0.000886,0.000052,0.007022,0.001577,0.004431,0.001402,blank sol
2016-05-19 23:23:00,0,0,0.001297,0.000990,0.007782,0.001803,0.004992,0.001571,0.003399,0.000079,...,0.000007,0.001296,0.000046,0.000983,0.000064,0.007763,0.001776,0.004960,0.001574,blank sol


In [11]:
df_peak = pd.DataFrame(columns = ["date", "sample", "peak_center_mass"])
data_1_corr = pd.DataFrame()
for sample in blk_ls:
    df = NU_data_read(path_all, sample, cup_config)
    datafile = "Results_" + str(sample) + ".txt"
    sample_name = df.extract_metadata(sample, "Sample Name:")
    date = df.extract_metadata(sample, "Date and Time:")
    peak_center = df.extract_metadata(sample, "Axial Mass(es) used:")
    data_1_corr["date"] = datetime.strptime(date, '%d/%m/%Y%H:%M')
    data_1_corr["sample"] = sample_name
    data_1_corr["peak_center_mass"] = peak_center
    df_peak = pd.concat([df_peak,data_1_corr], axis=0, ignore_index=True)

df_peak

UnboundLocalError: local variable 'metadata' referenced before assignment

In [14]:
df_all.to_csv(path_sav + "blanks.csv", mode='a', header=False)

In [215]:
df_internal_sub.to_csv(path_sav + "Sn_H8_L4_2_cycles_internal_norm.csv", mode='a', header=False)

In [229]:
df_internal_norm_mean_sd.to_csv(path + "Sn_H8_L4_2_cycles_internal_norm_mean_no_bgd_corr.csv")

In [389]:
epsilon.to_csv(path + "Sn_H8_L4_2_cycles_internal_norm_epsilon_bgd_corr.csv")